In [2]:
import pandas as pd
from multiprocessing import Pool, cpu_count
import numpy as np
from src.input.write_files import write_int_matrix_file
from src.input.read_files import find_files, numbers_from_file_name, normal_sec_size_int_from_file_name, number_from_text
from datetime import datetime
from src.algorithm.vanilla.circuit_synthesis import synthesise_circuit
from src.algorithm.vanilla.execution import load_matrix_and_benchmark
import csv

print(f'There are {cpu_count()} threads available on this machine.')

There are 12 threads available on this machine.


In [53]:
sample_size = 60
lower_size = 5
upper_size = 250 ##2047
step_size = 1

sizes = np.arange(lower_size, upper_size + step_size, step_size)
samples = np.arange(1, sample_size + 1, 1)

input_gen_processors = 10

input_dir = r'resources\section_size_benchmark\input_matrices'
input_file_prefix = 'SingMat'

benchmark_processors = 10

output_dir = r'resources\section_size_benchmark'
output_file_prefix = 'SectionSizeBenchmark'

output_file_fieldnames = ['size', 'run', 'sec_size', 'num_gates', 'process_time', 'initial_rss', 'final_rss']

In [54]:
input_gen_argument_tuples = ([(size, size * size, rf'..\{input_dir}\{input_file_prefix}_{size}_{sample}.txt') for size in sizes for sample in samples])

[print(argument_tuple) for argument_tuple in input_gen_argument_tuples[:min(10, len(input_gen_argument_tuples))]]

(5, 25, '..\\resources\\section_size_benchmark\\input_matrices\\SingMat_5_1.txt')
(5, 25, '..\\resources\\section_size_benchmark\\input_matrices\\SingMat_5_2.txt')
(5, 25, '..\\resources\\section_size_benchmark\\input_matrices\\SingMat_5_3.txt')
(5, 25, '..\\resources\\section_size_benchmark\\input_matrices\\SingMat_5_4.txt')
(5, 25, '..\\resources\\section_size_benchmark\\input_matrices\\SingMat_5_5.txt')
(5, 25, '..\\resources\\section_size_benchmark\\input_matrices\\SingMat_5_6.txt')
(5, 25, '..\\resources\\section_size_benchmark\\input_matrices\\SingMat_5_7.txt')
(5, 25, '..\\resources\\section_size_benchmark\\input_matrices\\SingMat_5_8.txt')
(5, 25, '..\\resources\\section_size_benchmark\\input_matrices\\SingMat_5_9.txt')
(5, 25, '..\\resources\\section_size_benchmark\\input_matrices\\SingMat_5_10.txt')


[None, None, None, None, None, None, None, None, None, None]

In [55]:
print(f'Generating {len(input_gen_argument_tuples)} matrices in size range from {lower_size} to {upper_size} ...')

if __name__ ==  '__main__':
    with Pool(processes = input_gen_processors) as pool:
        pool.starmap(write_int_matrix_file, input_gen_argument_tuples)

print(f'Saved matrices to {input_dir} ...')

Generating 14760 matrices in size range from 5 to 250 ...
Saved matrices to resources\section_size_benchmark\input_matrices ...


In [56]:
file_regex = rf'({input_file_prefix}).*\.txt'
size_regex = r'_\d+_'
sample_regex = r'_\d+\.'

input_file_names = find_files(rf'..\{input_dir}', file_regex)
input_file_names.sort(key = lambda input_file_name: numbers_from_file_name(input_file_name, size_regex, sample_regex))

[print(input_file_name) for input_file_name in input_file_names[:min(10, len(input_file_names))]]

SingMat_5_1.txt
SingMat_5_2.txt
SingMat_5_3.txt
SingMat_5_4.txt
SingMat_5_5.txt
SingMat_5_6.txt
SingMat_5_7.txt
SingMat_5_8.txt
SingMat_5_9.txt
SingMat_5_10.txt


[None, None, None, None, None, None, None, None, None, None]

In [57]:
file_name_arguments = [(number_from_text(input_file_name, size_regex),
                        number_from_text(input_file_name, sample_regex),
                        input_file_name)
                       for input_file_name in input_file_names]
file_name_arguments_df = pd.DataFrame(file_name_arguments, columns=['size', 'sample', 'path'])
file_name_arguments_df.head()

,size,sample,path
0,5,1,SingMat_5_1.txt
1,5,2,SingMat_5_2.txt
2,5,3,SingMat_5_3.txt
3,5,4,SingMat_5_4.txt
4,5,5,SingMat_5_5.txt


In [58]:
section_sizes_hard = [2,3,4,5]

benchmark_tuples_list = [(size, sec_size, sample, path) for size, sample, path in file_name_arguments_df.values.tolist() for sec_size in section_sizes_hard]

[print(benchmark_tuple) for benchmark_tuple in benchmark_tuples_list[:min(10, len(benchmark_tuples_list))]]

(5, 2, 1, 'SingMat_5_1.txt')
(5, 3, 1, 'SingMat_5_1.txt')
(5, 4, 1, 'SingMat_5_1.txt')
(5, 5, 1, 'SingMat_5_1.txt')
(5, 2, 2, 'SingMat_5_2.txt')
(5, 3, 2, 'SingMat_5_2.txt')
(5, 4, 2, 'SingMat_5_2.txt')
(5, 5, 2, 'SingMat_5_2.txt')
(5, 2, 3, 'SingMat_5_3.txt')
(5, 3, 3, 'SingMat_5_3.txt')


[None, None, None, None, None, None, None, None, None, None]

In [59]:
benchmark_argument_tuples = [(synthesise_circuit,
                              size,
                              sec_size,
                              sample,
                              rf'..\{input_dir}\{input_file_name}')
                             for (size, sec_size, sample, input_file_name) in benchmark_tuples_list]

In [60]:
output_file_timestamp = datetime.now().strftime('%d%m%y_%H%M')

print(f'Benchmarking runtime performance in size range from {lower_size} to {upper_size} in {benchmark_processors} threads ...')

if __name__ ==  '__main__':
    with Pool(processes = benchmark_processors) as pool:
        result_tuples = pool.starmap(load_matrix_and_benchmark, benchmark_argument_tuples)

print(f'Saving results to {output_dir} ...')

with open(rf'..\{output_dir}\{output_file_prefix}_{output_file_timestamp}.csv', mode='w', newline='') as output_file:
    writer = csv.DictWriter(output_file, fieldnames=output_file_fieldnames)
    writer.writeheader()

    for (size, run, sec_size, num_gates, process_time, initial_rss, final_rss) in result_tuples:

        writer.writerow({'size': size, 'run': run, 'sec_size': sec_size, 'num_gates': num_gates, 'process_time': process_time, 'initial_rss': initial_rss, 'final_rss': final_rss})

print('Done.')




Benchmarking runtime performance in size range from 5 to 250 in 10 threads ...
Saving results to resources\section_size_benchmark ...
Done.
